In [1]:
import math
import scipy as sp
import matplotlib.pyplot as plt
from scipy.stats import bernoulli, uniform, chi2
import numpy as np
from scipy.stats.mstats import gmean
from numpy.testing import assert_allclose
from utils import sprt_mart, alpha_mart, shrink_trunc, stratum_selector, multinomial_selector, get_global_pvalue
np.random.seed(123456789)

In [2]:
theta = 1/2 
hand_tally = np.concatenate((np.ones(40), np.zeros(40), np.ones(60), np.zeros(40)))
reported_tally = np.concatenate((np.ones(40), np.zeros(40), np.ones(70), np.zeros(30)))
omega = reported_tally - hand_tally
strata = np.concatenate((np.ones(80), 2*np.ones(100)))


v = np.array([2 * np.mean(reported_tally[strata == 1]) - 1, 2 * np.mean(reported_tally[strata == 2]) - 1])
u = 2 / (2 - v)
stratum_1 = (1 - omega[strata == 1]) / (2 - v[0])
stratum_2 = (1 - omega[strata == 2]) / (2 - v[1])

In [3]:
get_global_pvalue(strata = [stratum_1, stratum_2], u = u, v = v, rule = multinomial_selector)

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 8.20351745e-01, 6.08537263e-01, 4.83073919e-01,
       4.83073919e-01, 3.82440112e-01, 3.01931655e-01, 2.37695047e-01,
       1.96095207e-01, 1.84056938e-01, 1.72624764e-01, 1.30833950e-01,
       1.06157373e-01, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
      